# Importing and Checking Data

## Packages

In [ ]:
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import json

## Import Data and Specify Chunksize:

Here we set nrows (i.e. chunksize) = 5000

In [ ]:
checkin = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json', lines=True, nrows = 5000)
review = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json', lines=True, nrows = 5000)
business = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json', lines=True, nrows = 5000)
tip = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True, nrows = 5000)
user= pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json', lines=True, nrows = 5000)

## Check in 

In [ ]:
checkin.info()

In [ ]:
checkin.head()

Add checkin_id so it could be set as PK in SQL:

In [ ]:
checkin['checkin_id'] = range(1, len(checkin) + 1)

In [ ]:
earliest_date = str(checkin['date'].min())[:10]
latest_date = checkin['date'].max()

print("Check-in table earliest date:", earliest_date)
print("Check-in table latest date:", latest_date)

## Review

In [ ]:
review.head()

In [ ]:
review.info()

In [ ]:
earliest_date = review['date'].min()
latest_date = review['date'].max()

print("Review table earliest date:", earliest_date)
print("Review table latest date:", latest_date)

## Business

In [ ]:
business.head()

In [ ]:
business.info()

Here we adjust datatype of some columns so that the data format is consistent:

In [ ]:
if 'attributes' in business.columns:
    business['attributes'] = business['attributes'].astype(str)
else:
    print("Warning: 'attributes' column not found in the data.")

if 'hours' in business.columns:
    business['hours'] = business['hours'].astype(str)
else:
    print("Warning: 'hours' column not found in the data.")

## Tip

Add tip_id so it could be set as PK in SQL:

In [ ]:
tip['tip_id'] = range(1, len(tip) + 1)

In [ ]:
tip.head()

In [ ]:
tip.info()

In [ ]:
earliest_date = tip['date'].min()
latest_date = tip['date'].max()

print("Tip table earliest date:", earliest_date)
print("Tip table latest date:", latest_date)

## User

In [ ]:
user.head()

In [ ]:
user.info()

In [ ]:
earliest_date = user['yelping_since'].min()
latest_date = user['yelping_since'].max()

print("User table earliest date:", earliest_date)
print("User table latest date:", latest_date)

# Connecting to Azure SQL Server

In [ ]:
!pip install PyMySql
!pip install pyodbc
import sqlalchemy
from sqlalchemy import create_engine, types, inspect

In [ ]:
!apt-get update
!apt-get install -y unixodbc unixodbc-dev

# Add Microsoft's signing key and repository for SQL Server ODBC Driver
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | tee /etc/apt/sources.list.d/mssql-release.list

# Update the package list again after adding Microsoft's repository
!apt-get update

# Install Microsoft SQL Server ODBC Driver 17
!ACCEPT_EULA=Y apt-get install -y msodbcsql17

### Connection Details

In [ ]:
username = 'uradmin-name'
password = 'urpasswd'
server = 'urservername'
database = 'urdb'

### Setup Connection

In [ ]:
connection_string = f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

### Test Connection

In [ ]:
try:
    with engine.connect() as connection:
        print("Connection success!")
except Exception as e:
    print("Failed to connect:", e)

## to.sql()

### Business

In [ ]:
business.to_sql(
    name="business",
    con=engine,
    if_exists='replace',
    index=True,
    dtype={'attributes': sqlalchemy.types.NVARCHAR, 'hours': sqlalchemy.types.NVARCHAR}
)

### Review

In [ ]:
review.to_sql(
    name="review",
    con=engine,
    if_exists='replace',
    index=True
)

### Checkin

In [ ]:
checkin.to_sql(
    name="checkin",
    con=engine,
    if_exists='replace',
    index=True
)

### Tip

In [ ]:
tip.to_sql(
    name="tip",
    con=engine,
    if_exists='replace',
    index=True
)

### User

In [ ]:
user.to_sql(
    name="user",
    con=engine,
    if_exists='replace',
    index=True
)

## Execute SQL Syntax

In [ ]:
!pip install pyodbc sqlalchemy ipython-sql

In [ ]:
%load_ext sql

%sql $connection_string

## Check and Set PKs/FKs

* In this part, to ensure that every table in the database is well-structured, we need to examine each table to check for PKs and FKs

p.s: In this part, if setting is not working in the notebook, you can directly execute these codes **in Azure SQL Server**

First, Check all tables to see FKs and PKs:

In [ ]:
inspector = inspect(engine)

tables = inspector.get_table_names()

for table in tables:
    print(f"Table: {table}")
    
    pk = inspector.get_pk_constraint(table)
    print("Primary Key(s):", pk['constrained_columns'])
    
    fks = inspector.get_foreign_keys(table)
    print("Foreign Keys:", fks)

Please note that these are the very default settings, so it’s normal for them to be empty (unless you have already configured them in Azure)

To set keys, choose either methods:

1. Run Commands in **Server's Query**;

2. Use **"connection"** method to run commands separately

## Method 1: Directly Run in Azure SQL Server:

The following code should be executed in: **Azure Database - Query editor (preview) - Login - Query**

### Business

In [ ]:
%%sql
ALTER TABLE business
DROP CONSTRAINT IF EXISTS PK_business; 

--set pk:
ALTER TABLE business
ALTER COLUMN business_id VARCHAR(50) NOT NULL; 

--set fk:
ALTER TABLE business
ADD CONSTRAINT PK_business PRIMARY KEY (business_id); 

### Checkin

In [ ]:
%%sql
ALTER TABLE checkin
ALTER COLUMN checkin_id INT NOT NULL;
ALTER TABLE checkin
ALTER COLUMN business_id VARCHAR(50) NULL;

--set pk:
ALTER TABLE checkin
ADD CONSTRAINT PK_checkin PRIMARY KEY (checkin_id);

--set fk:
ALTER TABLE checkin
ADD CONSTRAINT FK_checkin_business FOREIGN KEY (business_id) REFERENCES business(business_id);

### Review

In [ ]:
%%sql
ALTER TABLE review
ALTER COLUMN review_id VARCHAR(50) NOT NULL;
ALTER TABLE review
ALTER COLUMN business_id VARCHAR(50) NULL;
ALTER TABLE review
ALTER COLUMN user_id VARCHAR(50) NULL;

--set pk
ALTER TABLE review
ADD CONSTRAINT PK_review PRIMARY KEY (review_id);

--set fk
ALTER TABLE review
ADD CONSTRAINT FK_review_business FOREIGN KEY (business_id) REFERENCES business(business_id);
ALTER TABLE review
ADD CONSTRAINT FK_review FOREIGN KEY (user_id) REFERENCES [user](user_id);

### Tips

In [ ]:
%%sql
ALTER TABLE tip
ALTER COLUMN tip_id INT NOT NULL;
ALTER TABLE tip
ALTER COLUMN business_id VARCHAR(50) NULL;
ALTER TABLE tip
ALTER COLUMN user_id VARCHAR(50) NULL;

--set pk:
ALTER TABLE tip
ADD CONSTRAINT PK_tip PRIMARY KEY (tip_id);

--set fk:
ALTER TABLE tip
ADD CONSTRAINT FK_tip FOREIGN KEY (business_id) REFERENCES business(business_id);

### User

In [ ]:
%%sql
ALTER TABLE [user]
ALTER COLUMN user_id VARCHAR(50) NOT NULL;

--set pk:
ALTER TABLE [user]
ADD CONSTRAINT PK_user PRIMARY KEY (user_id);

To check if the PKs and FKs are being set properly:

In [ ]:
%%sql
SELECT kcu.COLUMN_NAME AS ColumnName
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kcu
ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.TABLE_NAME = 'business' AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

## Method 2: engine.begin()

### Business

In [ ]:
from sqlalchemy import text
# Begin transaction
with engine.begin() as connection:
    try:
        # Ensure the column type meets pk requirements
        alter_column_sql = text("ALTER TABLE business ALTER COLUMN business_id VARCHAR(50) NOT NULL")
        connection.execute(alter_column_sql)
        
        # set pk
        add_pk_sql = text("ALTER TABLE business ADD CONSTRAINT PK_business PRIMARY KEY (business_id)")
        connection.execute(add_pk_sql)

        print("All set!")
    
    except Exception as e:
        # If an error occurs, the transaction will roll back
        print("An error occurred, transaction rolled back:", e)

To check if it has been set properly：

In [ ]:
%%sql
SELECT kcu.COLUMN_NAME AS ColumnName
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kcu
ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.TABLE_NAME = 'business' AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

### Checkin

In [ ]:
with engine.begin() as connection:
    try:
        # Drop the existing foreign key constraint if it exists
        drop_fk_checkin = text("ALTER TABLE checkin DROP CONSTRAINT IF EXISTS FK_checkin_business")
        connection.execute(drop_fk_checkin)
        
        # Alter the columns
        alter_checkin_id = text("ALTER TABLE checkin ALTER COLUMN checkin_id INT NOT NULL")
        connection.execute(alter_checkin_id_sql)
        
        alter_business_id = text("ALTER TABLE checkin ALTER COLUMN business_id VARCHAR(50) NULL")
        connection.execute(alter_business_id_sql)
        
        # Set pk
        add_pk_checkin = text("ALTER TABLE checkin ADD CONSTRAINT PK_checkin PRIMARY KEY (checkin_id)")
        connection.execute(add_pk_checkin)
        
        print("All set!")
    
    except Exception as e:
        print("An error occurred, transaction rolled back:", e)

In [ ]:
%%sql
SELECT kcu.COLUMN_NAME AS ColumnName
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kcu
ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.TABLE_NAME = 'checkin' AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

Here, I've only set the (PK) because the checkin table would have a referential integrity violation problem if a FK were set:

In [ ]:
%%sql
SELECT business_id 
FROM checkin 
WHERE business_id NOT IN (SELECT business_id FROM business)

However, if you wish to resolve this problem, you can either 

1. delete those rows，

or 

2. add the missing entries to business, 

as shown in the commented-out code below:

In [ ]:
#with engine.begin() as connection:
    #connection.execute(text("""
        #INSERT INTO business (business_id)
        #SELECT DISTINCT business_id
        #FROM checkin
        #WHERE business_id NOT IN (SELECT business_id FROM business)
    #"""))
    #print("Placeholder entries added to business.")

### Review

In [ ]:
with engine.begin() as connection:
    try:
        # Alter columns to ensure they meet requirements
        alter_review_id = text("ALTER TABLE review ALTER COLUMN review_id VARCHAR(50) NOT NULL")
        connection.execute(alter_review_id)

        alter_business_id = text("ALTER TABLE review ALTER COLUMN business_id VARCHAR(50) NULL")
        connection.execute(alter_business_id)

        alter_user_id = text("ALTER TABLE review ALTER COLUMN user_id VARCHAR(50) NULL")
        connection.execute(alter_user_id)

        # Set PK
        add_pk_review = text("ALTER TABLE review ADD CONSTRAINT PK_review PRIMARY KEY (review_id)")
        connection.execute(add_pk_review)
        
        print("All set!")
    except Exception as e:
        print("An error occurred, transaction rolled back:", e)

In [ ]:
%%sql
SELECT kcu.COLUMN_NAME AS ColumnName
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kcu
ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.TABLE_NAME = 'review' AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

Same problem when trying to set FKs:

In [ ]:
%%sql
SELECT business_id 
FROM review 
WHERE business_id NOT IN (SELECT business_id FROM business)

### Tip

In [ ]:
with engine.begin() as connection:
    try:
        # Alter the columns
        alter_tip_id = text("ALTER TABLE tip ALTER COLUMN tip_id INT NOT NULL")
        connection.execute(alter_tip_id)

        alter_business_id = text("ALTER TABLE tip ALTER COLUMN business_id VARCHAR(50) NULL")
        connection.execute(alter_business_id)

        alter_user_id = text("ALTER TABLE tip ALTER COLUMN user_id VARCHAR(50) NULL")
        connection.execute(alter_user_id)

        # Set PK
        add_pk_tip = text("ALTER TABLE tip ADD CONSTRAINT PK_tip PRIMARY KEY (tip_id)")
        connection.execute(add_pk_tip)

        print("All set!")

    except Exception as e:
        print("An error occurred, transaction rolled back:", e)

In [ ]:
%%sql
SELECT kcu.COLUMN_NAME AS ColumnName
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kcu
ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.TABLE_NAME = 'tip' AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

In [ ]:
%%sql
SELECT business_id 
FROM tip 
WHERE business_id NOT IN (SELECT business_id FROM business)

### User

In [ ]:
with engine.begin() as connection:
    try:
        # Alter the column to set user_id as NOT NULL
        alter_user_id = text("ALTER TABLE [user] ALTER COLUMN user_id VARCHAR(50) NOT NULL")
        connection.execute(alter_user_id)

        # Set primary key on user_id
        add_pk_user = text("ALTER TABLE [user] ADD CONSTRAINT PK_user PRIMARY KEY (user_id)")
        connection.execute(add_pk_user)

        print("All set!")

    except Exception as e:
        print("An error occurred, transaction rolled back:", e)

In [ ]:
%%sql
SELECT kcu.COLUMN_NAME AS ColumnName
FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc
JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kcu
ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE tc.TABLE_NAME = 'user' AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

## After Setting PKs:

To check all PKs in our database:

In [ ]:
inspector = inspect(engine)

tables = inspector.get_table_names()

for table in tables:
    print(f"Table: {table}")

    pk = inspector.get_pk_constraint(table)
    print("Primary Key(s):", pk['constrained_columns'])

    fks = inspector.get_foreign_keys(table) # I set nothing so it should be empty
    for fk in fks:
        print("Foreign Key:", fk['constrained_columns'], "->", fk['referred_table'], fk['referred_columns'])
    print("\n")

**--- End of this Notebook ---**